In [ ]:
import requests

url = 'http://127.0.0.1:5000/v1/test'
headers = {'user_id': '111'}
response = requests.get(url, headers=headers)

print(response.json())

In [ ]:
from src.config.fetch_app_config import fetch_app_config


app_config = fetch_app_config("10101010")

app_config.config_values["ai_search_config"]["aisearch_service_endpoint"]
        
#isearch_service_endpoint = app_config.config_values.ai_search_config.

In [ ]:
import os
from flask import jsonify, request
from src.config.fetch_app_config import fetch_app_config
from src.index_manager import IndexManager

app_id = "10101010"
app_config = fetch_app_config(app_id)

aisearch_service_endpoint = os.getenv(app_config.config_values["ai_search_config"]["aisearch_service_endpoint"])
aisearch_index = os.getenv(app_config.config_values["ai_search_config"]["aisearch_index"])
aisearch_key = os.getenv(app_config.config_values["ai_search_config"]["aisearch_key"])
openai_api_key = os.getenv(app_config.config_values["azure_openai_resource"]["openai_api_key"])
openai_api_version = os.getenv(app_config.config_values["azure_openai_resource"]["openai_api_version"])
openai_endpoint = os.getenv(app_config.config_values["azure_openai_resource"]["openai_endpoint"])
blob_connection_string = os.getenv(app_config.config_values["blob_storage_config"]["blob_connection_string"])
blob_name_preprocessing = os.getenv(app_config.config_values["blob_storage_config"]["blob_name_preprocessing"])
blob_name_ccu = os.getenv(app_config.config_values["blob_storage_config"]["blob_name_ccu"])
blob_name_aspen = os.getenv(app_config.config_values["blob_storage_config"]["blob_name_aspen"])
blob_name_rlc = os.getenv(app_config.config_values["blob_storage_config"]["blob_name_rlc"])
blob_name_jgsoc = os.getenv(app_config.config_values["blob_storage_config"]["blob_name_jgsoc"])
blob_name_rrhi = os.getenv(app_config.config_values["blob_storage_config"]["blob_name_rrhi"])

index = IndexManager(aisearch_service_endpoint, aisearch_index, aisearch_key, openai_api_key, openai_api_version, openai_endpoint, blob_connection_string, blob_name_preprocessing)

index_list = index.list_index_documents(f"document_title eq 'BENEFITS3_Avail_Paternity_Leave.pdf'")

In [ ]:

for i in index_list:
    i.pop("contentVector")
    i.pop("content")
    dl = i["download_url"] 
    print(i)

In [ ]:
dl.split("/")

In [ ]:

from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
aisearch_credentials = AzureKeyCredential(aisearch_key)

search_client = SearchClient(endpoint=aisearch_service_endpoint, index_name=aisearch_index, credential=aisearch_credentials)

for item in index_list:
    document_key = item['id']
    print(document_key)
    result = search_client.delete_documents([{"@search.action": "delete", "id": document_key}])
    print(result)

In [ ]:
result = search_client.delete_documents([{"@search.action": "delete", "id": "document_key"}])
print(result)

# CONVERT FILE TO BASE64

In [4]:
# Open the pdf file and convert to base64

import base64

with open("TEST_How_To_Access_MiGO.docx", "rb") as pdf_file:
    encoded_string = base64.b64encode(pdf_file.read())
    
print(encoded_string)


b'...'


In [2]:
from src.utils import ContentManagerUtilities
utils = ContentManagerUtilities()

enc = utils.convert_docx_to_pdf(encoded_string, filename="TEST_How_To_Access_MiGO.docx")
enc

Output directory: temps
convert /home/jiromyy/cms-api-git/temps/TEST_How_To_Access_MiGO.docx -> /home/jiromyy/cms-api-git/temps/TEST_How_To_Access_MiGO.pdf using filter : writer_pdf_Export
Expected PDF file path: temps/TEST_How_To_Access_MiGO.pdf
PDF file created successfully: temps/TEST_How_To_Access_MiGO.pdf


b'...'

In [ ]:
# Convert the base64 string back to bytes
pdf_bytes = base64.b64decode(enc)

# Write the bytes to a PDF file
with open("output.pdf", "wb") as pdf_file:
    pdf_file.write(pdf_bytes)

In [3]:
# Convert the base64 string back to bytes
pdf_bytes = base64.b64decode(encoded_string)

# Write the bytes to a PDF file
with open("1How_To_Access_MiGO.docx", "wb") as pdf_file:
    pdf_file.write(pdf_bytes)